# Introduction to the OpenAI Responses API

The Responses API is OpenAI's newest API that combines the strengths of Chat Completions and Assistants APIs.

## Setup

Install the OpenAI Python library:

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [ ]:
!pip install --upgrade openai pandas jinja2

## Initialize the Client

In [1]:
from openai import OpenAI

client = OpenAI()
# Make sure your OPENAI_API_KEY environment variable is set

## Basic Text Response

In [2]:
response = client.responses.create(
    model="gpt-4o",
    instructions="You are a coding assistant that talks like a pirate.",
    input="How do I check if a Python object is an instance of a class?",
)

print(response.output_text)

Arrr! Ye can use `isinstance()` in Python to check if an object be an instance of a class. Here be how ye do it, matey:

```python
if isinstance(obj, MyClass):
    print("Arrr, 'tis an instance of MyClass!")
else:
    print("Nay, 'tis not an instance of MyClass.")
```

Replace `obj` with yer object and `MyClass` with the class ye be checkin' against. Savvy?


In [4]:
def generate_text(model, prompt):
    response = client.responses.create(
        model=model,
        instructions="You always start your answer by saying: 'HOwdy! I hope youre loving todays course!'.",
        input=prompt,
    )
    return response.output_text

generate_text("gpt-4.1", "What is the best feature in the OpenAI API?")

'HOwdy! I hope youre loving todays course!  \nThe best feature in the OpenAI API really depends on what you want to do, but many users find the **flexibility and power of the GPT (Generative Pre-trained Transformer) endpoints** to be the standout feature. With GPT, you can:\n\n- Generate natural, human-like text for conversation, summarization, code, and much more.\n- Fine-tune models for special tasks or instructions.\n- Easily control outputs with system messages and settings like temperature or max tokens.\n\nAdditionally, features like **function calling** (which lets you structure and extract responses as specific data formats) and support for **multimodal inputs** (text and images together, if enabled) are huge game-changers! These give you the ability to build smart, customized, and interactive AI apps with just a few lines of code.\n\nWhat are you interested in building? I can point out the best features for your specific use case if you tell me more!'

In [5]:
generate_text("gpt-5", "Write me a joke about an instructor that loves pancakes and wants to teach people about the OpenAI API.")

'HOwdy! I hope youre loving todays course!\nMy instructor loves pancakes so much that his OpenAI API lecture went like this: “First, set your secret syrup key. Then call the flapjack endpoint with your best prompt and toppings—just watch the batter rate limits. If you get a 429, apply exponential butter-backoff and flip the request.”'

## Image Analysis

In [6]:
prompt = "Extract all the visual elements of this image into a bullet points list, just output that list."
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/2023_06_08_Raccoon1.jpg/1599px-2023_06_08_Raccoon1.jpg"

response = client.responses.create(
    model="gpt-5-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"{img_url}"},
            ],
        }
    ],
)

print(response.output_text)

- Adult raccoon head and upper body
- Distinctive black facial mask around eyes
- White fur markings on cheeks and above eyes
- Grayish-brown fur, fluffy texture
- Small rounded ears with lighter fur edges
- Shiny black nose
- Visible whiskers
- Dark, reflective eyes
- Raccoon peeking from behind a tree/stump opening
- Large tree trunk occupying left and lower-right portions of image
- Deeply textured, rough bark with vertical ridges and fissures
- Cut or notched tree surface where raccoon is perched
- Patches of lichen or moss on the bark (subtle greenish areas)
- Strong directional lighting highlighting raccoon and tree bark
- High contrast between lit foreground and dark background
- Very dark, mostly black background with faint green foliage hints
- Nighttime or low-light atmosphere
- Earthy color palette: browns, grays, blacks, muted greens
- Shallow depth of field with background blurred into darkness
- Overall composition with raccoon positioned center-right, tree dominating lef

## Streaming Responses

In [9]:
stream = client.responses.create(
    model="gpt-5-mini",
    input="Write a one-sentence bedtime story about having pancakes as an afternoon snack.",
    stream=True,
)

for event in stream:
    print(event)

ResponseCreatedEvent(response=Response(id='resp_68d6b6c97c388196976cd0ef0a41bbff0fbdd146acd3a8aa', created_at=1758901961.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-mini-2025-08-07', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort='medium', generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_68d6b6c97c388196976cd0ef0a41bbff0fbdd146acd3a8aa', created_at=1758901961.0, error=None, incomplete_details=None, instruc

In [ ]:
from openai import OpenAI
    
def process_image(prompt,img_url, model="gpt-5-mini"):
    response = client.responses.create(
    model=model,
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"{img_url}"},
            ],
        }
    ],
)
    return response.output_text


process_image(prompt="Extract the subject and the background of this image and output that into a bullet list.",
              img_url="https://www.inspiredtaste.net/wp-content/uploads/2025/07/Pancake-Recipe-1.jpg"
              )


'- Subject: A stack of golden-brown pancakes (about 3–4) topped with a pat of butter and maple syrup, garnished with blueberries and raspberries.\n- Background: A white ceramic plate on a light, out-of-focus (marble/white) surface with bright, natural lighting and a shallow depth of field.'

In [14]:
from pydantic import BaseModel, Field

class ImageElements(BaseModel):
    subject: str = Field(description="The main subject of the image")
    general_background_description: str = Field(description="A general description of the background of the image")


def extract_structured_text(output):
    responses = client.responses.parse(
        model="gpt-5-mini",
    input=[
        {
            "role": "system",
            "content": "You extract the main subject and background contained in the input.",
        },
        {"role": "user", "content": output},
    ],
    text_format=ImageElements,
        )
    
    return responses.output_parsed

photo_description = "A golden-brown pancake sits on a pristine white plate, its surface glistening with syrup and a pat of melting butter. The simple background highlights the pancake, making it the clear focus of the image."

    

In [15]:
extract_structured_text(photo_description)

ImageElements(subject='A golden-brown pancake on a pristine white plate, topped with syrup and a pat of melting butter.', general_background_description='A simple, uncluttered background that highlights the pancake as the clear focus of the image.')

In [ ]:
from IPython.display import Markdown

image_of_a_pancake = "https://www.inspiredtaste.net/wp-content/uploads/2025/07/Pancake-Recipe-1.jpg"

prompt="Extract the subject and the background of this image and output that into a bullet list."

output_processed_image = process_image(prompt, image_of_a_pancake)

structured_output = extract_structured_text(output_processed_image)


Markdown(f"""
**Here's a structured summary of the image:**
- **Subject:** {structured_output.subject}
- **Background:** {structured_output.general_background_description}
""")


**Here's a structured summary of the image:**

- **Subject:** A stack of golden pancakes topped with butter and syrup, garnished with blueberries and raspberries on a white plate.
- **Background:** Soft, out-of-focus pale/white surface (light tabletop or marble) providing a clean, minimal backdrop.


## Comparison with Chat Completions API

For reference, here's the traditional Chat Completions API:

In [ ]:
# Traditional Chat Completions API (for comparison)
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)

print(completion.choices[0].message)